In [1]:
import pandas as pd
import numpy as np
import pickle as pk

import pickle as pk
import xgboost as xgb
import matplotlib.pyplot as plt

from scipy.stats import gaussian_kde
import h2o
from h2o.automl import H2OAutoML
from h2o.tree import H2OTree

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle("../../data/df.pkl")

### IA Filter

In [3]:
df = df.loc[df.type_bool == True]
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN10005,"[[10.027000000001863, 4.996, 8.081], [33.94500...","[[0.015000000006693881, 0.2977, 2.206], [18.94...","[[0.0, 1.033, 1.325], [13.015000000006694, 0.4...","[[0.03500000000349246, 1.826, 3.029], [48.875,...","[0.0, 1.0381919191919462, 2.0763838383838924, ...","([10.027000000001863, 10.782565656567513, 11.5...","([0.015000000006693881, 1.0530404040470573, 2....","([0.0, 1.0281717171717606, 2.056343434343521, ...","([0.03500000000349246, 1.0222929292964054, 2.0...","[2.000004411755352, 2.000003547336676, 2.00000...",...,-26.554909,15.768490,16.436792,17.114795,-20.292951,-24.222566,-11.295562,Ia,True,1.08327
SN10007,"[[0.0, 6.266, 10.48], [10.105999999999767, 17....","[[0.03200000000651926, 7.161, 9.313], [2.01600...","[[2.0, 1.24, 2.429], [10.121000000006461, 20.0...","[[0.9770000000062282, 0.6586, 5.329], [15.9930...","[0.0, 1.5136565656566285, 3.027313131313257, 4...","([0.0, 1.5132626262626456, 3.0265252525252913,...","([0.03200000000651926, 1.5451717171782005, 3.0...","([2.0, 3.49314141414145, 4.9862828282829, 6.47...","([0.9770000000062282, 2.480787878794107, 3.984...","[11.341091517194192, 8.812204685737349, 6.9798...",...,-99.877031,21.670852,24.155364,-26.622112,-27.616185,-19.981432,49.299250,Ia,True,0.82331
SN100361,"[[6.957000000002154, 16.02, 23.92], [7.9690000...","[[0.023000000001047738, 4.903, 3.324], [6.9840...","[[0.0, 0.7103, 2.626], [5.9760000000023865, 5....","[[8.055000000000291, 2.567, 2.665], [15.035000...","[0.0, 1.3031111111111275, 2.606222222222255, 3...","([6.957000000002154, 8.15910101010318, 9.36120...","([0.023000000001047738, 1.315494949496019, 2.6...","([0.0, 1.3026767676768058, 2.6053535353536117,...","([8.055000000000291, 9.27674747474778, 10.4984...","[8.233549869515556, 9.464488874691614, 10.4511...",...,102.074783,-95.727930,4.861555,-29.749073,23.442440,5.441237,12.503390,Ia,True,0.90012
SN100471,"[[0.0, 3.79, 4.764], [11.969000000004598, 1.67...","[[0.023999999997613486, 2.202, 3.146], [3.0429...","[[0.008000000001629815, 0.4567, 2.129], [3.004...","[[0.046999999998661224, 6.92, 2.486], [3.07500...","[0.0, 1.4638181818181601, 2.9276363636363203, ...","([0.0, 1.3621414141414037, 2.7242828282828073,...","([0.023999999997613486, 1.4875757575733735, 2....","([0.008000000001629815, 1.3701818181834458, 2....","([0.046999999998661224, 1.3283737373723963, 2....","[5.63245418874034, 5.603687474301877, 5.594769...",...,-40.831628,28.629443,12.801879,-0.555542,13.065768,-21.776767,9.358561,Ia,True,0.92640
SN100490,"[[0.0, 0.01278, 2.833], [8.11699999999837, 1.7...","[[0.026999999994586688, 1.849, 5.434], [8.1449...","[[15.0, 0.4107, 4.054], [46.042999999997846, 6...","[[1.1679999999978463, 1.572, 6.291], [49.96899...","[0.0, 0.7267979797979933, 1.4535959595959866, ...","([0.0, 0.5955656565656131, 1.1911313131312262,...","([0.026999999994586688, 0.7435858585804729, 1....","([15.0, 15.565262626262582, 16.130525252525164...","([1.1679999999978463, 1.8829999999978817, 2.59...","[8.351889035545303, 8.47302530763294, 8.594179...",...,-36.484191,38.259352,-88.907842,76.656123,50.003481,-5.379145,-57.678266,Ia,True,0.73814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN98918,"[[0.0, 3.693, 2.686], [4.984000000004016, 18.7...","[[5.94100000000617, 6.403, 9.191], [13.1840000...","[[4.996000000006461, 8.241, 6.049], [12.141000...","[[0.027000000001862645, 3.092, 4.045], [5.0120...","[0.0, 1.3522323232323885, 2.704464646464777, 4...","([0.0, 1.3017676767676767, 2.6035353535353534,...","([5.94100000000617, 7.233222222228395, 8.52544...","([4.996000000006461, 6.297686868693313, 7.5993...","([0.027000000001862645, 1.2480808080826955, 2....","[16.23491946702271, 16.82763060047037, 17.3637...",...,-34.255294,-4.973908,11.915895,-56.738013,-27.269352,9.479893,7.803001,Ia,True,0.76083
SN99404,"[[0.0, 6.586, 10.48], [15

In [4]:
X = df.loc[:,'f1':'REDSHIFT_SPEC']
y = df.loc[:,'REDSHIFT_SPEC']

### Auto ML

In [21]:
h2o.init(ip="localhost", port=54322)

Checking whether there is an H2O instance running at http://localhost:54322 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.9.1" 2020-11-04; OpenJDK Runtime Environment (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.9.1+1-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /home/fmoliveira/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp3oyartz4
  JVM stdout: /tmp/tmp3oyartz4/h2o_fmoliveira_started_from_python.out
  JVM stderr: /tmp/tmp3oyartz4/h2o_fmoliveira_started_from_python.err
  Server is running at http://127.0.0.1:54326
Connecting to H2O server at http://127.0.0.1:54326 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,1 month and 19 days
H2O_cluster_name:,H2O_from_python_fmoliveira_lri8wg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.881 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


### Preparing different size DFs

In [22]:
divisions = [500,1000,1500,2000,2500,3000,3500,4000,4500,5078]
amls = {}
for i in divisions:
    aml = H2OAutoML(max_models=20, seed=1)
    aml.train(x=list(df.loc[:,'f1':'f20'].columns), y='REDSHIFT_SPEC', training_frame=h2o.H2OFrame(X.iloc[0:i]))
    print(aml.leaderboard.as_data_frame(use_pandas=True))
    amls[f'size_{i}'] = aml


Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
                                             model_id  mean_residual_deviance  \
0    StackedEnsemble_AllModels_AutoML_20201116_235016                0.009922   
1   StackedEnsemble_BestOfFamily_AutoML_20201116_2...                0.009967   
2                        GBM_2_AutoML_20201116_235016                0.010381   
3                        GBM_4_AutoML_20201116_235016                0.010488   
4                        GBM_3_AutoML_20201116_235016                0.010525   
5          GBM_grid__1_AutoML_20201116_235016_model_1                0.010598   
6                        GBM_1_AutoML_20201116_235016                0.010772   
7          GBM_grid__1_AutoML_20201116_235016_model_2                0.010835   
8                        XRT_1_AutoML_20201116_235016                0.010990   
9                        DRF

21  0.214139  0.045856  0.168514  0.134992  
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
                                             model_id  mean_residual_deviance  \
0    StackedEnsemble_AllModels_AutoML_20201117_000127                0.008486   
1   StackedEnsemble_BestOfFamily_AutoML_20201117_0...                0.008581   
2                        GBM_3_AutoML_20201117_000127                0.008844   
3                        GBM_4_AutoML_20201117_000127                0.008887   
4                        GBM_1_AutoML_20201117_000127                0.009037   
5                        GBM_2_AutoML_20201117_000127                0.009075   
6          GBM_grid__1_AutoML_20201117_000127_model_1                0.009318   
7          GBM_grid__1_AutoML_20201117_000127_model_2                0.009430   
8                        XRT_1_AutoML_20201117_000127           

21  0.207850  0.043202  0.158985       NaN  
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
                                             model_id  mean_residual_deviance  \
0    StackedEnsemble_AllModels_AutoML_20201117_001821                0.008124   
1   StackedEnsemble_BestOfFamily_AutoML_20201117_0...                0.008207   
2                        GBM_2_AutoML_20201117_001821                0.008385   
3                        GBM_4_AutoML_20201117_001821                0.008451   
4                        GBM_3_AutoML_20201117_001821                0.008452   
5                        GBM_1_AutoML_20201117_001821                0.008628   
6          GBM_grid__1_AutoML_20201117_001821_model_2                0.008651   
7                        XRT_1_AutoML_20201117_001821                0.008820   
8          GBM_grid__1_AutoML_20201117_001821_model_1           

21  0.197056  0.038831  0.153967       NaN  
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
                                             model_id  mean_residual_deviance  \
0    StackedEnsemble_AllModels_AutoML_20201117_003753                0.007562   
1   StackedEnsemble_BestOfFamily_AutoML_20201117_0...                0.007697   
2                        GBM_2_AutoML_20201117_003753                0.007775   
3                        GBM_3_AutoML_20201117_003753                0.007808   
4                        GBM_4_AutoML_20201117_003753                0.007826   
5                        GBM_1_AutoML_20201117_003753                0.007942   
6          GBM_grid__1_AutoML_20201117_003753_model_2                0.007995   
7                        GBM_5_AutoML_20201117_003753                0.008153   
8          GBM_grid__1_AutoML_20201117_003753_model_1           

In [23]:
amls

{'size_500': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b5576d8>,
 'size_1000': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b5567b8>,
 'size_1500': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b54aef0>,
 'size_2000': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b531cc0>,
 'size_2500': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b51d668>,
 'size_3000': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b51f358>,
 'size_3500': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b524b38>,
 'size_4000': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b5703c8>,
 'size_4500': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b569e48>,
 'size_5078': <h2o.automl.autoh2o.H2OAutoML at 0x7f006b5b8198>}

In [27]:
amls['size_500'].leader.rmse()

0.045183633264755

In [30]:
for key in amls.keys():
    print(amls[key].leader.rmse())

0.045183633264755
0.04560952820293093
0.04251948173609046
0.04492073316284886
0.04083489231002861
0.044077153525424705
0.048364026143644365
0.045937701802140705
0.0479126270366597
0.049769048794606065
